In [1]:
import time
from datetime import timedelta
import pandas as pd

import numpy as np
from sklearn.neighbors import NearestNeighbors

import math


#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

## 2.Récupération et analyse du jeu de données

In [2]:
doc=pd.read_csv('jena_climate_2009_2016.csv')
print(doc)

                  Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  \
0       01.01.2009 00:00:00    996.50     -8.00    264.76        -8.92   
1       01.01.2009 00:10:00    996.52     -8.02    265.40        -8.90   
2       01.01.2009 00:20:00    996.57     -8.41    265.01        -9.28   
3       01.01.2009 00:30:00    996.53     -8.51    264.91        -9.31   
4       01.01.2009 00:40:00    996.51     -8.31    265.12        -9.07   
...                     ...       ...       ...       ...          ...   
420547  31.12.2016 23:20:00   1000.07     -4.05    269.10        -8.13   
420548  31.12.2016 23:30:00    999.93     -3.35    269.81        -8.06   
420549  31.12.2016 23:40:00    999.82     -3.16    270.01        -8.21   
420550  31.12.2016 23:50:00    999.81     -4.23    268.94        -8.53   
420551  01.01.2017 00:00:00    999.82     -4.82    268.36        -8.42   

        rh (%)  VPmax (mbar)  VPact (mbar)  VPdef (mbar)  sh (g/kg)  \
0        93.30          3.33          3.

In [3]:
print(doc.info())
print(doc.shape)
print(doc.head())
print(doc.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420552 entries, 0 to 420551
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date Time        420552 non-null  object 
 1   p (mbar)         420552 non-null  float64
 2   T (degC)         420552 non-null  float64
 3   Tpot (K)         420552 non-null  float64
 4   Tdew (degC)      420552 non-null  float64
 5   rh (%)           420552 non-null  float64
 6   VPmax (mbar)     420552 non-null  float64
 7   VPact (mbar)     420552 non-null  float64
 8   VPdef (mbar)     420552 non-null  float64
 9   sh (g/kg)        420552 non-null  float64
 10  H2OC (mmol/mol)  420552 non-null  float64
 11  rho (g/m**3)     420552 non-null  float64
 12  wv (m/s)         420552 non-null  float64
 13  max. wv (m/s)    420552 non-null  float64
 14  wd (deg)         420552 non-null  float64
dtypes: float64(14), object(1)
memory usage: 48.1+ MB
None
(420552, 15)
             Date 

## 3.Préparation et transformation du jeu de données

In [4]:
doc['Date Time'] = pd.to_datetime(doc['Date Time'], format="%d.%m.%Y %H:%M:%S")

In [5]:
doc.set_index("Date Time")

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:00:00,996.50,-8.00,264.76,-8.92,93.30,3.33,3.11,0.23,1.96,3.17,1307.75,1.03,1.75,142.7
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
2016-12-31 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
2016-12-31 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2


In [6]:
print(doc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420552 entries, 0 to 420551
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date Time        420552 non-null  datetime64[ns]
 1   p (mbar)         420552 non-null  float64       
 2   T (degC)         420552 non-null  float64       
 3   Tpot (K)         420552 non-null  float64       
 4   Tdew (degC)      420552 non-null  float64       
 5   rh (%)           420552 non-null  float64       
 6   VPmax (mbar)     420552 non-null  float64       
 7   VPact (mbar)     420552 non-null  float64       
 8   VPdef (mbar)     420552 non-null  float64       
 9   sh (g/kg)        420552 non-null  float64       
 10  H2OC (mmol/mol)  420552 non-null  float64       
 11  rho (g/m**3)     420552 non-null  float64       
 12  wv (m/s)         420552 non-null  float64       
 13  max. wv (m/s)    420552 non-null  float64       
 14  wd (deg)         420

In [7]:
print(doc)

                 Date Time  p (mbar)  T (degC)  Tpot (K)  Tdew (degC)  rh (%)  \
0      2009-01-01 00:00:00    996.50     -8.00    264.76        -8.92   93.30   
1      2009-01-01 00:10:00    996.52     -8.02    265.40        -8.90   93.30   
2      2009-01-01 00:20:00    996.57     -8.41    265.01        -9.28   93.40   
3      2009-01-01 00:30:00    996.53     -8.51    264.91        -9.31   93.90   
4      2009-01-01 00:40:00    996.51     -8.31    265.12        -9.07   94.20   
...                    ...       ...       ...       ...          ...     ...   
420547 2016-12-31 23:20:00   1000.07     -4.05    269.10        -8.13   73.10   
420548 2016-12-31 23:30:00    999.93     -3.35    269.81        -8.06   69.71   
420549 2016-12-31 23:40:00    999.82     -3.16    270.01        -8.21   67.91   
420550 2016-12-31 23:50:00    999.81     -4.23    268.94        -8.53   71.80   
420551 2017-01-01 00:00:00    999.82     -4.82    268.36        -8.42   75.70   

        VPmax (mbar)  VPact

In [8]:
#Normalisation Min-Max : Nouvelle valeur = (valeur – min) / (max – min)
#Normalisation moyenne : Nouvelle valeur = (valeur – moyenne) / (écart type)

#Min-max : 
doc=(doc-doc.min())/(doc.max()-doc.min())
print(doc)


C:\Users\amand\AppData\Local\Temp\ipykernel_10308\1875516641.py:5: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  doc=(doc-doc.min())/(doc.max()-doc.min())


       Date Time  p (mbar)  T (degC)  Tpot (K) Tdew (degC)    rh (%)  \
0            0.0  0.814742  0.248963  0.233125    0.334372  0.923033   
1       0.000002  0.814939  0.248632  0.243662    0.334788  0.923033   
2       0.000005   0.81543  0.242163  0.237241    0.326891  0.924182   
3       0.000007  0.815037  0.240504  0.235594    0.326268  0.929925   
4        0.00001   0.81484  0.243822  0.239052    0.331255  0.933372   
...          ...       ...       ...       ...         ...       ...   
420547   0.99999  0.849828   0.31448  0.304577     0.35079  0.690982   
420548  0.999993  0.848452  0.326091  0.316266    0.352244  0.652039   
420549  0.999995  0.847371  0.329242  0.319559    0.349127  0.631361   
420550  0.999998  0.847273  0.311494  0.301943    0.342477  0.676048   
420551       1.0  0.847371  0.301708  0.292394    0.344763   0.72085   

       VPmax (mbar) VPact (mbar) VPdef (mbar) sh (g/kg) H2OC (mmol/mol)  \
0          0.037886     0.084272     0.004999  0.082813     

## 4.Extraction des caractéristiques

## 5.Recherche par similarité

#### 5.1.Méthode exacte

#### 5.2.Méthode approximative

## 6.Exploitation pratique des résultats de recherche